In [1]:
from relationGraph import Relation, RelationGraph, MatrixOfRelationGraph
from autoencoder import seedy, AutoEncoder
import utilityFunctions as uf
from main import test_build_relation_graph_with_symertic_data, test_convert_graph_to_2D_matrix, test_get_matix_for_autoencoder, test_autoencoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from base import load_source
from os.path import join
import numpy as np

from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply, Masking, Dropout, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import autoencoder as ae


# from scipy.stats import norm

In [2]:
num = 81
num = 162
num = 242
num = 323
# list_of_nums = [81, 162, 242, 323, 403, 483, 564]
list_of_nums = [81]
# num = 1614
batch_size = 100
original_dim = np.power(num, 2)
# intermediate_dim = int(original_dim / 3.)
output_dim = np.power(num, 2)
latent_dim = 20
intermediate_dim = 250
nb_epoch = 10
epsilon_std = .1
n_packs = 200
epochs = 300

f = np.load('/data/samples/dicty/' + str(num) + '_org_data.npz')
data = f[f.files[0]]
fn = '/data/samples/dicty/' + str(num) + '_data.npz'
log_dir = '/data/logs/'


#np.random.seed(1337)  # for reproducibility

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

def vae_loss2(x, x_decoded_mean):
    xent_loss = original_dim * categorical_crossentropy(x,  x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

def vae_loss(input_img, output):
    # compute the average MSE error, then scale it up, ie. simply sum on all axes
    reconstruction_loss = K.sum(K.square(output - input_img))
    # compute the KL loss
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.square(K.exp(z_log_var)), axis=-1)
    # return the average loss over all images in batch
    total_loss = K.mean(reconstruction_loss + kl_loss)    
    return total_loss

for num in list_of_nums:
    
    batch_size = 100
    original_dim = np.power(num, 2)
    # intermediate_dim = int(original_dim / 3.)
    output_dim = np.power(num, 2)
    latent_dim = 20
    intermediate_dim = 250
    nb_epoch = 10
    epsilon_std = .1
    n_packs = 100
    epochs = 300

    f = np.load('/data/samples/dicty/' + str(num) + '_org_data.npz')
    data = f[f.files[0]]
    fn = '/data/samples/dicty/' + str(num) + '_data.npz'
    log_dir = '/data/logs/'

    #Encoding Layers
    x = Input(batch_shape=(batch_size, original_dim))
    # x = Masking(mask_value=0, input_shape=(original_dim, ))(x)
    h = Dense(intermediate_dim,activation="elu")(x)
    h= Dropout(0.7)(h)
    h = Dense(intermediate_dim, activation='elu')(h)
    h=BatchNormalization(axis=0)(h)
    h = Dense(intermediate_dim, activation='elu')(h)

    #Latent layers
    z_mean=Dense(latent_dim)(h)
    z_log_var=Dense(latent_dim)(h)
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    #Decoding layers 

    decoder_1= Dense(intermediate_dim, activation='elu')
    decoder_2=Dense(intermediate_dim, activation='elu')
    decoder_2d=Dropout(0.7)
    decoder_3=Dense(intermediate_dim, activation='elu')
    decoder_out=Dense(output_dim, activation='sigmoid')
    x_decoded_mean = decoder_out(decoder_3(decoder_2d(decoder_2(decoder_1(z)))))

    vae = Model(x, x_decoded_mean)

    #Potentially better results, but requires further hyperparameter tuning
    #optimizer=keras.optimizers.SGD(lr=0.005, momentum=0.001, decay=0.0, nesterov=False,clipvalue=0.05)
    vae.compile(optimizer="adam", loss=vae_loss,metrics=[
        "categorical_accuracy",
    #     "fmeasure",
    #     "top_k_categorical_accuracy",
        "mean_squared_error"
    ])

    vae.summary()

    callbacks = [
                TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True),
                EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto')
            ]

    vae.fit_generator(
        ae.data_generator([fn], n_packs), 
        steps_per_epoch=n_packs,
        callbacks=callbacks,
        epochs=epochs
    )
    
    vae.save('/data/vae/weights/' + str(num) + '_model.h5')
    

print('Finish!!!')

# f = np.load('/data/samples/dicty/' + str(num) + '_data.npz')
# data = f[f.files[0]]


# data.shape
# data.flatten().shape
# img = vae.predict(data.reshape(100, original_dim), batch_size=100)

# print(np.mean(img))
# print(np.mean(data))
# mse = mean_squared_error(img, data)
# print(mse)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (100, 6561)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (100, 250)           1640500     input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (100, 250)           0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (100, 250)           62750       dropout[0][0]                    
__________________________________________________________________________________________________
batch_norm

ResourceExhaustedError: OOM when allocating tensor with shape[100,250] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dropout_1/cond/dropout/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dropout_1/cond/dropout/Shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics/categorical_accuracy/Mean/_187 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1638_metrics/categorical_accuracy/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oo

In [6]:
# num = 81
list_of_nums = [81, 162, 242, 323, 403, 483, 564]
for num in list_of_nums:
    print('--------------' + str(num) + '--------------')
    original_dim = np.power(num, 2)
    fn = '/data/samples/dicty/' + str(num) + '_org_data.npz'
    f = np.load(fn)
    # test_data = np.asarray([f[f.files[0]].flatten()])
    # test_data = np.asarray([data.flatten()])

    test_data = f[f.files[0]]

    # data.flatten().shape
    # test_data = vae.predict(data.reshape(100, original_dim), batch_size=100)

    # prediction with normal data

    vae = load_model('/data/vae/weights/' + str(num) + '_model.h5',  custom_objects={'vae_loss': vae_loss})

    y = vae.predict(test_data.reshape(100, original_dim), batch_size=100)
    # y = model.predict(test_data)
#     print(test_data.shape)
#     print(y.shape)
    mse = mean_squared_error(test_data, y)
#     print(test_data[0])
#     print(y[0])
#     print()
#     print(test_data.shape)
#     print(y.shape)
#     print()
    print('MSE org data: ' + str(mse))
    print()


--------------81--------------
MSE org data: 0.009575274675055903

--------------162--------------
MSE org data: 0.009483858867600163

--------------242--------------


ResourceExhaustedError: OOM when allocating tensor of shape [250,58564] and type float
	 [[Node: training_11/Adam/zeros_18 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [250,58564] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training_11/Adam/zeros_18', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_fo

In [25]:
num = 81    # most densely filled  0.05
num = 162   # most densely filled  0.1
num = 323   # most densely filled  0.2
# num = 483   # most densely filled  0.3
# num = 645   # most densely filled  0.4
# num = 807   # most densely filled  0.5


# original_dim = 784
# intermediate_dim = 256
original_dim = np.power(num, 2)
intermediate_dim = 250
latent_dim = 20
batch_size = 100
epochs = 50
epsilon_std = .1
n_packs = batch_size

f = np.load('/data/samples/dicty/' + str(num) + '_org_data.npz')
data = f[f.files[0]]
fn = '/data/samples/dicty/' + str(num) + '_data.npz'

log_dir = '/data/logs/'


def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)


class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs


decoder = Sequential([
    Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
    Dense(original_dim, activation='sigmoid')
])

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)

z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(stddev=epsilon_std,
                                   shape=(K.shape(x)[0], latent_dim)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

x_pred = decoder(z)

vae = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='rmsprop', loss=nll, metrics=["mean_squared_error"])
vae.summary()

callbacks = [
            TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True),
            EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto')
        ]

vae.fit_generator(
    ae.data_generator(fn, n_packs), 
    steps_per_epoch=n_packs,
    callbacks=callbacks,
    epochs=epochs
)


f = np.load(fn)
data = f[f.files[0]]


data.shape
data.flatten().shape
img = vae.predict(data.reshape(100, original_dim), batch_size=100)

print(np.mean(img))
print(np.mean(data))
mse = mean_squared_error(img, data)
print(mse)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 104329)       0                                            
__________________________________________________________________________________________________
dense_144 (Dense)               (None, 250)          26082500    input_37[0][0]                   
__________________________________________________________________________________________________
dense_145 (Dense)               (None, 20)           5020        dense_144[0][0]                  
__________________________________________________________________________________________________
dense_146 (Dense)               (None, 20)           5020        dense_144[0][0]                  
__________________________________________________________________________________________________
kl_diverge

100/100 [==============================] - 18s 182ms/step - loss: nan - mean_squared_error: 0.0390
0.22505543
0.22031216628761538
0.038938773334643746
